# COMP90051 Project 1

##### Author: Tingsheng Lai 781319 (Tinson)

##### Date Created: 19/08/2018

## Extract Training Data

In [1]:
from networkx import DiGraph

G = DiGraph()
with open("train.txt", 'r') as f:
    for line in f.read().split('\n'):
        line = [int(i) for i in line.split()]
        if line:
            G.add_edges_from((line[0], i) for i in line[1:])

## Util Functions

In [2]:
import math
from networkx.algorithms.simple_paths import all_simple_paths

def neighbour_features(p1, p2):
    ns1, ns2 = set(G.successors(p1)), set(G.successors(p2))
    lns1, lns2 = len(ns1), len(ns2)
    cn = ns1.intersection(ns2)
    lcn = len(cn)
    def _helper(x):
        lsx = G.degree(x)
        return 1 / lsx if lsx > 0 else 0
    # (Common, Jaccard, Adamic/Adar, Preferential)
    return lcn, lcn / len(ns1.union(ns2)), sum(_helper(i) for i in cn), lns1 * lns2

def katz(p1, p2, beta=0.5, cutoff=2):
    pls = {}
    for path in all_simple_paths(G, p1, p2, cutoff):
        lp = len(path)
        pls[lp] = pls.get(lp, 0) + 1
    return sum(beta**l * c for l, c in pls.items())

## Model
### Initialisation

In [8]:
from keras import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(100, input_shape=(6,)))
model.add(Dense(100))
model.add(Dense(1, activation="sigmoid"))
model.compile("sgd", "mean_squared_error")

### Training

In [11]:
import math
import numpy as np
from networkx.algorithms.shortest_paths.unweighted import single_source_shortest_path_length

for src in G:
    spls = single_source_shortest_path_length(G, src)
    nodes = list(G)
    ln = len(nodes)
    curr = 0
    while curr < ln:
        data = np.array([(-spls.get(dest, 0), *neighbour_features(src, dest), katz(src, dest), dest in G.successors(src)) for dest in nodes[curr:curr + 64]])
        model.fit(data[:, :-1], data[:, -1], epochs=2)
        curr += 64

In [21]:
# from networkx.algorithms.shortest_paths.weighted import bidirectional_dijkstra

# with open("test-public.txt") as f:
#     lines = f.read().split('\n')
#     for line in lines[1:]:
#         if line:
#             pid, p1, p2 = [int(i) for i in line.split()]
#             features = bidirectional_dijkstra(G, p1, p2)[0], *neighbour_features(p1, p2), katz(p1, p2)
#             print(f"{pid}: {features} {model.predict(np.array([features]))}")

1: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
2: (2, 0, 0.0, 0, 0, 0.25) [[0.00548263]]
3: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
4: (2, 5, 0.053763440860215055, 0.020287496541954657, 1176, 0.5) [[5.3243605e-19]]
5: (3, 6, 0.018808777429467086, 0.006220485861005233, 15486, 0) [[0.]]
6: (2, 8, 0.026845637583892617, 0.020926719347548108, 2673, 1.0) [[0.]]
7: (2, 0, 0.0, 0, 0, 0.125) [[0.00565327]]
8: (2, 0, 0.0, 0, 0, 0.125) [[0.00565327]]
9: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
10: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
11: (2, 0, 0.0, 0, 0, 0.375) [[0.00531711]]
12: (2, 0, 0.0, 0, 0, 0.125) [[0.00565327]]
13: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
14: (2, 0, 0.0, 0, 0, 0.125) [[0.00565327]]
15: (2, 4, 0.002711864406779661, 0.010785728624617685, 264368, 0.25) [[0.]]
16: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
17: (2, 4, 0.001756697408871322, 0.0011884919012019937, 362748, 0.125) [[0.]]
18: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
19: (3, 0, 0.0, 0, 0, 0) [[0.00059378]]
20: (2, 0, 0.0, 0, 0, 0.25) [[0.00548263]]


NetworkXNoPath: No path between 2574465 and 1663756.